In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed

# Step 1: Load and preprocess the data
# Purpose: We are loading census tract data, points of interest (POI), and hazard data for Georgia to calculate hazard risk scores.
census_tract = gpd.read_file('/home/ojin/working_space/SIG/01. justice40/01. data/01. raw/01. Justice40/1.0-shapefile-codebook/filtered_usa/filtered_usa.shp')
poi = gpd.read_file('/your/shp/to/calculate/hazard/risk/poi.shp')
shapefile_path = r'/home/ojin/working_space/SIG/01. justice40/01. data/02. processed/Georgia_hazard_merged(abs).shp'
gdf = gpd.read_file(shapefile_path)

# Normalize the building loss rate and flood risk scores using MinMaxScaler (scaled between 0-1)
scaler = MinMaxScaler()
flood_avg_damage = 3.8  # Example average flood damage from USA disaster statistics
gdf['BLR_scaled'] = scaler.fit_transform(gdf[['BLR_abs']])  # Building loss rate (BLR) scaling
gdf['Fld_scaled'] = scaler.fit_transform(gdf[['Fld_abs']] * flood_avg_damage)  # Flood risk scaling

# Fill NaN values with 0
gdf[['BLR_scaled', 'Fld_scaled']] = gdf[['BLR_scaled', 'Fld_scaled']].fillna(0)

# Calculate the total hazard score by averaging building loss rate and flood risk
gdf['total_cost'] = (gdf['BLR_scaled'] + gdf['Fld_scaled']) / 2

# Ensure consistent projection (EPSG:3857) for all datasets
census_tract = census_tract.to_crs(epsg=3857)
poi = poi.to_crs(epsg=3857)
gdf = gdf.to_crs(epsg=3857)

# Step 2: Calculate hazard risk scores for each POI
# Purpose: Use the buffer areas around each POI to calculate area-weighted risk scores based on the intersecting census tracts.
radius_A = 1000  # 1km buffer for nearby hazards
radius_B = 3000  # 3km buffer for broader hazards
poi_buffers_A = poi.buffer(radius_A)
poi_buffers_B = poi.buffer(radius_B)

# Function to calculate area-weighted hazard risk scores for each POI
def calculate_risk_based_score_polygon(poi_buffer, census_tracts, risk_scores_scaled):
    intersecting_tracts = census_tracts[census_tracts.intersects(poi_buffer)]
    if not intersecting_tracts.empty:
        weights = intersecting_tracts.intersection(poi_buffer).area / intersecting_tracts.area
        weighted_risk = np.average(risk_scores_scaled[intersecting_tracts.index], weights=weights)
        return weighted_risk
    return 0  # Return 0 if no intersecting tracts are found

# Calculate risk scores for building loss rate (BLR) and flood risk
risk_based_scores_A_scaled = Parallel(n_jobs=5)(delayed(calculate_risk_based_score_polygon)(
    poi_buffers_A[i], gdf, gdf['BLR_scaled'].to_numpy()) for i in range(len(poi)))

risk_based_scores_B_scaled = Parallel(n_jobs=5)(delayed(calculate_risk_based_score_polygon)(
    poi_buffers_B[i], gdf, gdf['Fld_scaled'].to_numpy()) for i in range(len(poi)))

# Step 3: Assign hazard risk scores to the POI dataset and scale the final scores
# Purpose: Aggregate the BLR and flood risk scores, and rescale the final hazard score between 0-1.
aggregate_scores_scaled = (np.array(risk_based_scores_A_scaled) + np.array(risk_based_scores_B_scaled)) / 2
poi['hazard_risk'] = aggregate_scores_scaled
poi['hazard_risk_scaled'] = scaler.fit_transform(poi[['hazard_risk']])

# Save the updated POI shapefile with the hazard risk scores
output_shapefile_path = '/home/ojin/working_space/SIG/03. Data/01. Processed/02. filtered/poi_filtered/osm_poi_filterd_with_hazard_scaled.shp'
poi.to_file(output_shapefile_path)
print(f"POIs with hazard risk scores saved to: {output_shapefile_path}")

# Step 4: Visualization of hazard risk scores
# Purpose: Visualize the POIs with color intensity based on their hazard risk score.
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
poi.plot(column='hazard_risk_scaled', cmap='OrRd', markersize=10, legend=True, ax=ax, edgecolor='black')

# Add title and labels to the plot
plt.title("POIs with Hazard Risk Scores (Scaled)", fontsize=15)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Display the plot
plt.show()
